<a href="https://colab.research.google.com/github/KA-Jones/Voluntary_REDD_Analysis_GEE/blob/master/Generate_Forest_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Voluntary REDD Analysis (Google Earth Engine)

Generate annual time series measuring forest coverage in for all municipalities in the Brazilian Amazon. Annual land use/cover classifications from [MapBiomas](https://mapbiomas.org/) provide data between 1985 and 2019 using the Collection 5 dataset.



Developed by K.A. Jones, Department of Geography, San Diego State University


Based on Voluntary REDD Analysis developed by T. West.
[Github](https://github.com/thaleswest/Voluntary-REDD-analysis)

---

# Part 2. Generate Forest Time Series

###**Input:** Image Asset (Step 1) saved to Earth Engine Assets

###**Output:** One image corrected for probited land cover transitions

**REQUIRED:**

You must complete [PART 1: Data Processing](https://github.com/KA-Jones/Voluntary_REDD_Analysis_GEE/blob/master/Create_Annual_Forest_Asset.ipynb) before generating forest coverage time series.

-OR-

Have access to a previous Image Asset created as the result of using PART 1: Data Processing.

### Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro).

If you need to visualize intermediate or final results, uncomment install of `geehydro`. The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.

***Uncomment these lines if you are running this notebook for the first time.***

In [ ]:
# !pip install earthengine-api
# !pip install geehydro

###Import Earth Engine (ee), Authenticate, and initialize Earth Engine API
You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for the first time or if you are getting an authentication error. 

Uncomment lines to import foluim and geehydro if needing to visualize output.

In [ ]:
import ee
# import folium
# import geehydro

ee.Authenticate()
ee.Initialize()

### Define study years and import datasets.
Input: 

1.   Start and End Study Years (Collection 4: 1985-2018 & Collection 5: 1985-2019)
2.   Access to Image Asset from Part 1: Data Processing (e.g. `REDD_MapBiomas_Col5_85_19`) 
3.   2015 Municipal boundaries in the Brazilian Amazon (808 total)


In [ ]:
## Define Study Years. Start Year and End Year (int)
startDate = 1985
endDate = 2019

## Define Year of Municipal Boundaries (string)
## Available annual boundaries: 2000, 2001, 2005, 2007, 2010, 2013, 2014, 2015
municipal_year = '2015'


## Load data
REDD_landcover = ee.Image('projects/Biggs-Lab/REDD_MapBiomas_Col5_85_19')  ## Created from MapBiomas Collection 5 (1985-2019)
municipalities = ee.FeatureCollection('projects/Biggs-Lab/Amazon_Municip_'+municipal_year)


# ## Uncomment ONLY IF performing analysis on municipalities in a specific Brazilian state, enter the corresponding state code.
# ## This value corresponds to the first two digits for a given municipality (e.g. '11' = Rondonia)

# ## Define State Code (string)
# state_code = 11

# ## Filter municipalities layer by municipalities in a given state.
# municip_table = municipalities.filterMetadata('STATE_CODE', 'equals', state_code)

### Define Functions

In [ ]:
## Applied in Step 1
def reclass_img_to_col(yr):
  """Function 'reclass_img_to_col' creates one annual image for each study year using the original
  MapBiomas landcover image. Pixel values in each annual image are reclassified from the MapBiomas
  categories to new landcover categories: forest, non-forest, water, and cloud/NA."""
  year = ee.Number(yr)
  class_yr = ee.String('Landcover_').cat(year.format("%d"))
  img = REDD_landcover.select(class_yr)
  img = img.set('YearID', year.format("%d"))    # Add new metadata property 'YearID' to be used in filtering collection or identify images by year
  img = img.set('date', year)                   # Add new metadata property 'date' to be used in filtering collection or identify images by year
  return img



## Applied in Step 2
def get_area(image):
    """Function 'get_area' computes the area (km2) of forest pixels in each image outside of
    reduce_region function so the process does not need to be repeated each time a new
    feature is used within 'reduceRegion'."""
    area = image.eq(1).multiply(ee.Image.pixelArea().divide(1000000))
    return area.copyProperties(image)



## Applied in Step 3
def reduce_regions(image):
  """Function 'reduce_regions' uses reduceRegions() with reducer: sum() to complute the
  total area (km2) of forested pixels in each municipality feature"""
  reduced = image.reduceRegions(collection = municip_table, reducer = ee.Reducer.sum(), scale = 30)
  reduced = reduced.filter(ee.Filter.neq('sum', None))
  reduced = reduced.map(lambda f: f.set('date', image.get('YearID')))
  return reduced



## Applied in Step 4 - Table Formatting (Triplets)
def format(table, rowID, colID):
  """Function 'format' defines a 2D table of rowId x colId from a table of triplets."""
  ## Get a FeatureCollection with unique row IDs.
  rows = table.distinct(rowID)
  ## Join the table to the unique IDs to get a collection in which each feature stores a list of all features having a common row ID.
  joined = ee.Join.saveAll('matches').apply(
      primary = rows,
      secondary = table,
      condition = ee.Filter.equals(
          leftField = rowID,
          rightField = rowID
      )
  )
  def unique_id(row):
    values = ee.List(row.get('matches'))
    def col_id_val(feat):
      feature = ee.Feature(feat)
      id_val = [feature.get(colID), feature.get('sum')]
      return id_val
    values = values.map(col_id_val)
    return row.select([rowID]).set(ee.Dictionary(values.flatten())) 
  return joined.map(unique_id)



## Applied in Step 5
def add_properties(feature):
  """Function 'add_properties' adds the metadata properties of the original municipality feature collection to output
  table based on the shared CD_GEOCMU id. The metadata properties are lost during joins and some mapped operations."""
  id = feature.get('CD_GEOCMU')
  match_munic = ee.Feature(municip_table.filterMetadata('CD_GEOCMU','equals',id).first())
  return feature.copyProperties(match_munic)


def update_properties(feature):
  old_names = feature.propertyNames()
  new_names = ['area_km2', 'municipal_name', 'state', 'municipal_code', 'state_code', 'system:index'] + [str(int) for int in list(range(startDate, (endDate+1)))]
  return feature.select(old_names, new_names)

### Create an image collection from the REDD_landcover image asset

In [ ]:
## Reclassify the original MapBiomas land cover classes for years 1985 - 2019
## Input: REDD_landcover

## Create list of study years to map over
years_list = ee.List.sequence(startDate, endDate)

###------###
## Step 1. Create image collection from band names
image_col = ee.ImageCollection.fromImages(ee.List(years_list.map(reclass_img_to_col)))


###------###
## Step 2. Apply function 'get_area' to compute forest coverage in km2.
final_col_area = image_col.map(get_area)


###------###
## Step 3. Sum the area of forest in each municipality. Use reduceRegions() and map() over Image Collection.
regions_result = final_col_area.map(reduce_regions)
regions_result = regions_result.flatten()

## Remove contents of '.geo' for clarity
results_out = regions_result.select(['.*'], None, False)


###------###
## Step 4. Format a table of triplets to organize output (columns: year, rows: Municipality Code (CD_GEOCMU)).
output_table = format(results_out, 'CD_GEOCMU', 'date');


###------###
## Step 5. Add original municipal properties to output table.
output_table_props = output_table.map(add_properties)

## Update property names
output_table_final = output_table_props.map(update_properties)

### Export Results

In [ ]:
## Define name of output and taskID:
output_task_id = 'Contemp_Municip_'+municipal_year


## Set export properites
properties = {'driveFolder': 'RFF',  'driveFileNamePrefix': output_task_id+'_Y'+str(startDate)+'_'+str(endDate), 'fileFormat': 'CSV'}

## Define the task in Python
task = ee.batch.Export.table(output_table_final, output_task_id+'_Y'+str(startDate)+'_'+str(endDate), properties)


# ## Uncomment if running for specific state (state code will populate file name)
# ## Set export properites
# properties = {'driveFolder': 'RFF',  'driveFileNamePrefix': output_task_id+state_code+'_Y'+str(startDate)+'_'+str(endDate), 'fileFormat': 'CSV'}

# ## Define the task in Python
# task = ee.batch.Export.table(output_table, output_task_id+str(state_code)+'_Y'+str(startDate)+'_'+str(endDate), properties)


## Start the task, equivalent to hit the "run" button in the editor
task.start()

### Create an interactive map to visualize results (Optional)
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [ ]:
## Set 'viz_layer' equal to desired image for visualization.
viz_layer = ee.Image(image_col.filterMetadata('date','equals',2000).first())

## Set starting location to center on Rhondonia
Map = folium.Map(location=[-11, -63], zoom_start=6)
Map.setOptions('ROADMAP')

## Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4,
  'palette': ['000000', '76B37F', 'F0E5AA', '03A5FC', 'FC03DB']}  #[Black, Olive, Beige, Blue, Fuchsia]


## Add selected layer to map.
Map.addLayer(test, vis_params, 'Landcover')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map